In [1]:
import os
import yaml
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.tools import StructuredTool, tool
from langchain_core.tools import ToolException
from typing import List, Optional, Dict, Any
from pprint import pprint
import warnings

warnings.filterwarnings("ignore")

In [2]:
@tool
def fake_tool():
    """This is a fake tool"""
    return "I am a fake tool"

In [3]:
def try_except_tool(self, func):
    try:
        return func
    except ToolException as e:
        print(f"Error: {e}")

def enhance_agent_tools(self, agent_tools: List):
    enhanced_agent_tools = [
        StructuredTool(
            name=tool.name,
            description=tool.description,
            args_schema=tool.args_schema,
            func=self.try_except_tool(tool._run),
            handle_tool_error=True,
            handle_validation_error=True
        ) for tool in agent_tools
    ]
    return enhanced_agent_tools

In [ ]:
class MyAgent:
    def __init__(self, tools, custom_prompt: str = None):
        self.llm = AzureChatOpenAI(
            model="gpt-4o",
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        )
        self.tools = tools
        self.agent_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.agent = create_tool_calling_agent(
            llm=self.llm,
            tools=self.enhance_agent_tools(tools),
            prompt=ChatPromptTemplate.from_messages([
                ("system", custom_prompt if custom_prompt else "You are an experienced assistant. You are able to perform any task asked by the user through all the tools you have access to. Your goal is to complete the user's task by using the tools you have access to. This is a core instruction: ALWAYS ALWAYS ALWAYS USE A STRING AS ID"),
                ("placeholder", "{chat_history}"),
                ("human", "{input}"),
                ("placeholder", "{agent_scratchpad}"),
            ]),
        )
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, memory=self.agent_memory, verbose=True)

    def myinvoke(self, input: str):
        if not isinstance(input, str):
            raise ValueError("Input must be a string")
        result = self.executor.invoke({"input": input})
        print(result)
        print("\n==================================================")
        print("--> Agent Response:")
        print(result['output'])
        print("\n==================================================")

    def try_except_tool(self, func):
        try:
            return func
        except ToolException as e:
            print(f"Error: {e}")

    def enhance_agent_tools(self, agent_tools: List):
        enhanced_agent_tools = [
            StructuredTool(
                name=tool.name,
                description=tool.description,
                args_schema=tool.args_schema,
                func=self.try_except_tool(tool._run),
                handle_tool_error=True,
                handle_validation_error=True
            ) for tool in agent_tools
        ]
        return enhanced_agent_tools

In [2]:
class MyAgent:
    def __init__(self, tools, custom_prompt: str = None):
        self.llm = AzureChatOpenAI(
            model="gpt-4o",
            api_key=os.getenv("AZURE_OPENAI_API_KEY"),
        )
        self.tools = tools
        self.agent_memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        self.agent = create_tool_calling_agent(
            llm=self.llm,
            tools=self.enhance_agent_tools(tools),
            prompt=ChatPromptTemplate.from_messages([
                ("system", custom_prompt if custom_prompt else "You are an experienced assistant. You are able to perform any task asked by the user through all the tools you have access to. Your goal is to complete the user's task by using the tools you have access to. This is a core instruction: ALWAYS ALWAYS ALWAYS USE A STRING AS ID"),
                ("placeholder", "{chat_history}"),
                ("human", "{input}"),
                ("placeholder", "{agent_scratchpad}"),
            ]),
        )
        self.executor = AgentExecutor(agent=self.agent, tools=self.tools, memory=self.agent_memory, verbose=True)

    def myinvoke(self, input: str):
        if not isinstance(input, str):
            raise ValueError("Input must be a string")
        result = self.executor.invoke({"input": input})
        print(result)
        print("\n==================================================")
        print("--> Agent Response:")
        print(result['output'])
        print("\n==================================================")

    def try_except_tool(self, func):
        try:
            return func
        except ToolException as e:
            print(f"Error: {e}")

    def enhance_agent_tools(self, agent_tools: List):
        enhanced_agent_tools = [
            StructuredTool(
                name=tool.name,
                description=tool.description,
                args_schema=tool.args_schema,
                func=self.try_except_tool(tool._run),
                handle_tool_error=True,
                handle_validation_error=True
            ) for tool in agent_tools
        ]
        return enhanced_agent_tools

In [3]:
@tool
def fake_tool():
    """This is a fake tool"""
    return "I am a fake tool"

agent = MyAgent(tools=[fake_tool])

In [4]:
agent.myinvoke("Hello")

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


BadRequestError: Error code: 400 - {'error': {'code': 'invalidPayload', 'message': "'prompt' is a required property", 'type': 'invalid_request_error'}}